# Analysis Of OT 2on2 Rocket League Matches


[10:26, 17.10.2019] Tobias Bey: Weil Zahn ziehen der beste Weg ist zu verteidigen und das
Spiel schnell zu machen - hört ruhig mehr auf den Carry Train :) und abwarten ist halt Schrott 😁

[11:04, 17.10.2019] Tobias Bey: Braucht man nicht für rekorde. Vll mal bisschen weniger driften Oli. Und mehr Tore mache 

[19:32, 17.10.2019] Tobias Bey: Ich bin die ganze Zeit unter Strom, spiele am Limit, da interessiert mich der super Sonic weniger. Ich versuche in den super super Sonic zu kommen. #legende
    
[00:39, 20.10.2019] Tobias Bey: Witzige Aktionen sind völlig fehl am Platz. Verrückte Aktionen mit Mehrheit zu positiven Ergebnissen sind der Schlüssel zum Erfolg. Klar funktioniert auch mal was nicht und es ist auch mal ein unglücklicher Rückpass oder nem querschläger dabei... aber ein paar spielentscheidende Traumtore löschen alles vorherige sofort und man ist direkt wieder etwas im plus.... #listentothedigital2on2legende
## Part 1: Games and Sessions

In [1]:
# Basic Python Libs
import os
import sys
import time
import datetime
import operator
import csv
import pydotplus
import plotly
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
from scipy import stats
from itertools import groupby, islice

In [2]:
# Define Global Variables for example
player_name_1 = ['Demigod_Hope', 'Cpt. Noobus Maximus']
player_name_2 = ['Cloudwalker', 'Al!en']
player_name_list = [player_name_1, player_name_2] 

team_name_1 = ['Cloudwalker & Demigod_Hope', 'Cloudwalker & Demigod_Hope & Cloudwalker']
team_name_list = [team_name_1] 

random_names = ['Randoms']
random_name_list = []

class Name:
    def __init__(self, name):
        self.name = name
        
player_class_list = []
for player_name in player_name_list:
    player_class_list.append(Name(player_name))

team_class_list = []
for team_name in team_name_list:
    team_class_list.append(Name(team_name))
    
random_class_list = []
for random_name in random_name_list:
    random_class_list.append(Name(random_name))

#map_played = 'Champions Field'
session_delay = np.timedelta64(1, 'h')
colorblind_friendly_list = ['#AE9C45', '#6073B1', '#A7B8F8', '#052955', '#b19660', 'black', 'darkblue']
width=0.2

# Paths
data_path = "Data\\"
plot_path = "Plots\\"

# Files
all_games_file = data_path+"OT_2on2.csv"
player_output = data_path+"Only_OT_2on2.csv"

# Data Imports
df_all_games = pd.read_csv(all_games_file, delimiter=',')

# Reference Dataframe Columns
df_columns = ['replay id', 'replay title', 'map', 'date', 'team name', 'player name', 'car id', 'car name',
              'score', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded', 
              'goals conceded while last defender', 'shooting percentage', 'bpm', 'avg boost amount', 
              'amount collected', 'amount collected big pads', 'amount collected small pads', 
              'count collected big pads', 'count collected small pads', 'amount stolen', 
              'amount stolen big pads', 'amount stolen small pads', 'count stolen big pads', 
              'count stolen small pads', '0 boost time', 'total distance', 'time slow speed', 
              'percentage slow speed', 'time boost speed', 'percentage boost speed', 
              'time supersonic speed', 'percentage supersonic speed', 'time on ground', 
              'percentage on ground', 'time low in air', 'percentage low in air', 'time high in air', 
              'percentage high in air', 'time powerslide', 'avg powerslide time', 'count powerslide', 
              'time most back', 'percentage most back', 'time most forward', 'percentage most forward', 
              'time in front of ball', 'percentage in front of ball', 'time behind ball', 
              'percentage behind ball', 'time in front of ball.1', 'percentage in front of ball.1', 
              'time defensive half', 'percentage defensive half', 'time offensive half', 
              'percentage offensive half', 'time defensive third', 'percentage defensive third', 
              'time neutral third', 'percentage neutral third', 'time offensive third', 
              'percentage offensive third', 'demos inflicted', 'demos taken']

# Features I want to add for the team
team_feature_list = ['goals', 'assists', 'saves', 'shots', 'bpm', 'amount collected', 'demos inflicted', 
                     'demos taken', 'amount collected big pads', 'amount collected small pads']

# Functions I use

# def session_feature_plot(class_list, feature, ylabel, fig_name, vertical_flip=False, 
#                          width=0.2, avg_line_flip=True, percentage=False, xlabel='Session'):
#     fig = plt.figure(figsize=(14,8))
#     ax = fig.add_subplot(111)
#     for k, class_item in enumerate(class_list):
#         feature_avg_list = []
#         feature_std_list = []
#         for j in range(len(sesson_index_list)-1):
#             feature_avg_list.append(class_item.df[feature][sesson_index_list[j]:sesson_index_list[j+1]].mean())
#             feature_std_list.append(class_item.df[feature][sesson_index_list[j]:sesson_index_list[j+1]].sem())
# #         if avg_line_flip == 'True':
# #             ax.axhline(player.df[feature].mean(), index_session[0]-1,index_session[-1]+1,
# #                         color=colorblind_friendly_list[k])
#         feature_avg = class_item.df[feature].mean()
#         feature_std = class_item.df[feature].std()
#         feature_stderr = class_item.df[feature].sem()
#         ax.axhline(feature_avg, index_session[0]-1, index_session[-1]+1, color=colorblind_friendly_list[k])
#         ax.bar(index_session+(k-1)*width, feature_avg_list, width, color=colorblind_friendly_list[k],  
#                yerr = feature_std_list, ecolor='black')
#     if percentage:
#         ax.set_ylim([0.0,100.0])
#         plt.locator_params(axis='y', nbins=11)
#     ymin_axes, ymax_axes = ax.get_ylim()
#     for k, class_item in enumerate(class_list):
#         feature_upper = class_item.df[feature].mean()+class_item.df[feature].sem()
#         feature_lower = class_item.df[feature].mean()-class_item.df[feature].sem()
#         ax.axvspan(xmin=index_session[0]-1, xmax=index_session[-1]+1,
#                    ymin=feature_lower/ymax_axes, ymax=feature_upper/ymax_axes,
#                 alpha=0.1, facecolor=colorblind_friendly_list[k])
#     ax.set_ylabel(ylabel)
#     ax.set_xlabel(xlabel)
#     plt.xticks(index_session+width/2, index_session)
#     ax.legend((class_list[0].name[0], 
#                class_list[1].name[0],
#                class_list[2].name[0],
#                class_list[3].name[0]),
#                loc='best')         
#     plt.tight_layout()
#     plt.savefig(plot_path+fig_name)
#     plt.show()
    
def session_feature_plot(class_list, feature, ylabel, fig_name, vertical_flip=False, 
                         width=0.2, avg_line_flip=True, percentage=False, xlabel='Session'):
    fig = plt.figure(figsize=(14,8))
    ax = fig.add_subplot(111)
    class_length = len(class_list)
    legend_name_list = []
    for k, class_item in enumerate(class_list):
        legend_name_list.append(class_item.name[0])
        feature_avg_list = []
        feature_std_list = []
        for session in range(min(class_item.df.session), max(class_item.df.session)+1):
            if len(class_item.df.session[class_item.df.session == session]) == 0:
                pass
            else:
                session_mean = class_item.df[feature][class_item.df.session == session].mean()
                session_std = class_item.df[feature][class_item.df.session == session].sem()
                feature_avg_list.append(session_mean)
                feature_std_list.append(session_std)
        feature_avg = class_item.df[feature].mean()
        feature_std = class_item.df[feature].std()
        feature_stderr = class_item.df[feature].sem()
        ax.axhline(feature_avg, index_session[0]-1, index_session[-1]+1, color=colorblind_friendly_list[k])
        ax.bar(index_session+(k-class_length+4)*width,
               feature_avg_list,
               width,
               color=colorblind_friendly_list[k],  
               yerr = feature_std_list, ecolor='black')
    if percentage:
        ax.set_ylim([0.0,100.0])
        plt.locator_params(axis='y', nbins=11)
    ymin_axes, ymax_axes = ax.get_ylim()
    for k, class_item in enumerate(class_list):
        feature_upper = class_item.df[feature].mean()+class_item.df[feature].sem()
        feature_lower = class_item.df[feature].mean()-class_item.df[feature].sem()
        ax.axvspan(xmin=index_session[0]-1, xmax=index_session[-1]+1,
                   ymin=feature_lower/ymax_axes, ymax=feature_upper/ymax_axes,
                alpha=0.1, facecolor=colorblind_friendly_list[k])
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    plt.xticks(index_session+width/2, index_session+1)
    ax.legend(legend_name_list,
               loc='best')         
    plt.tight_layout()
    plt.savefig(plot_path+fig_name)
    plt.show()    

def most_feature_plot(class_list, feature_list, fig_name, ylim_flip=False, vertical_flip=False,
                      width=0.2, logscale=False):
    fig = plt.figure(figsize=(14,8))
    ax = fig.add_subplot(111)
    ind_feature = np.arange(int(len(feature_list)))
    legend_name_list = []
    for k, class_item in enumerate(class_list):
        legend_name_list.append(class_item.name[0])
        most_features_list = []
        for feature in feature_list:
            most_features_list.append(class_item.df[feature].max())
            
        ax.bar(ind_feature+k*width, most_features_list, width, color=colorblind_friendly_list[k], 
               ecolor='black')
    if ylim_flip:
        ax.set_ylim([0.0,100.0])
        plt.locator_params(axis='y', nbins=11)
    if logscale:
        plt.yscale('log')
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        ax.yaxis.set_major_formatter(formatter)
    ax.set_ylabel('Score')
    ax.set_xlabel('')
    if ((len(feature_list) > 8) | (vertical_flip)):
        plt.xticks(ind_feature+width/2, feature_list, rotation='vertical')
    else:
        plt.xticks(ind_feature+width/2, feature_list)
    Title = "Highest Score in:"
    ax.set_title(Title)
    ax.legend(legend_name_list,
               loc='best')
    plt.tight_layout()
    plt.savefig(plot_path+fig_name)
    plt.show()
    
    
def value_in_a_row(df, feature_name, feature_in_a_row_name, threshold, add_value, operator=operator.eq):
    df[feature_in_a_row_name] = 0
    value_in_a_row_counter = 0
    for row in range(len(df)):
        if operator(df[feature_name][row],threshold):
            if add_value == 'df':
                value_in_a_row_counter += df[feature_name][row]
            else:
                value_in_a_row_counter += add_value
            df.loc[row, feature_in_a_row_name] = value_in_a_row_counter
        else:
            value_in_a_row_counter = 0
            
def difference(df, feature_name, feature_in_a_row_name, threshold, start_value, operator=operator.eq):
    df[feature_in_a_row_name] = 0
    k = start_value
    for row in range(len(df)-1):
        df.loc[row, feature_in_a_row_name] = k
        if operator(abs(df[feature_name][row]-df[feature_name][row+1]),threshold):
            k +=1
    df.loc[len(df)-1, feature_in_a_row_name] = k
            
def add_df_to_class(class_list):
    class_df_check = []
    if class_list == player_class_list:
        feature = 'player name'
    elif class_list == team_class_list:
        feature = 'team name'
    for class_item in class_list:
        class_item.df = df_cleaned[df_cleaned[feature].isin(class_item.name)].reset_index(drop='True')
        value_in_a_row(class_item.df, 'result', 'wins in a row', 1, 1, operator.eq)
        value_in_a_row(class_item.df, 'result', 'losses in a row', 0, 1, operator.eq)
        value_in_a_row(class_item.df, 'goals', 'goals in a row', 1, 'df', operator.ge)
        value_in_a_row(class_item.df, 'saves', 'saves in a row', 1, 'df', operator.ge)
        value_in_a_row(class_item.df, 'assists', 'assists in a row', 1, 'df', operator.ge)
        #difference(class_item.df, 'date', 'session', session_delay, 0, operator=operator.ge)
        class_item.df['session games'] = class_item.df.groupby('session')['session'].transform('count')
        class_df_check.append(len(class_item.df)) 
    
    if class_list == player_class_list:
        if len(np.unique(class_df_check)) > 1:
            print("Error: Something went wrong with the dataframe for this class assignment.")
        else:
            pass
    elif class_list == team_class_list:
        if len(np.unique(class_df_check)) > (len(team_class_list)/2):
            print("Error: Something went wrong with the dataframe for this class assignment.")
        else:
            pass
        
def session_feature_mean(class_list, feature, new_column_name, percentage=False):
    for class_item in class_list:
        class_item.session_winrate = []
        class_item.session_result = [0] * len(index_session)
        for session in range(min(class_item.df.session), max(class_item.df.session)+1):
            if percentage:
                session_feature_avg = 100*class_item.df[feature][class_item.df.session == session].mean()
            else:
                session_feature_avg = class_item.df[feature][class_item.df.session == session].mean()
            class_item.session_winrate.append(session_feature_avg)
            class_item.df[new_column_name][class_item.df.session == session] = session_feature_avg

def session_champ(class_list, store_list):
    winrate_dump_list = []
    for session in index_session:
        for class_item in class_list:
            winrate_dump_list.append(class_item.session_winrate[session])
        max_winrate_dump = max(winrate_dump_list)
        max_inds = [i for i, x in enumerate(winrate_dump_list) if x == max_winrate_dump]
        if len(max_inds) > 1:
            for max_index in max_inds:
                class_list[max_index].session_result[session] = 0.5
            session_champions_list.append("DRAW")
        else:
            class_list[max_inds[0]].session_result[session] = 1
            store_list.append(class_list[max_inds[0]].name[0])
        winrate_dump_list = []


In [4]:
# Manage Dataframe
df_all_games.sort_values(by=['date', 'team name'], inplace=True)
df_all_games = df_all_games.reset_index(drop=True)
df_sum_gb = df_all_games.groupby(['date', 'team name'], 
                                 as_index=False)[team_feature_list].sum()
df_sum_gb = pd.concat([df_sum_gb]*2)
df_sum_gb.sort_values(by=['date', 'team name'], inplace=True)
df_sum = df_sum_gb.reset_index(drop=True)

for feature in team_feature_list:
    team_feature = 'team ' + str(feature)
    df_all_games[team_feature] = df_sum[feature]

df_all_games['result'] = df_all_games.apply(lambda row: 1 if row['team goals'] > row['goals conceded'] 
                                            else 0, axis=1)
df_all_games['session winrate'] = 0

df_all_games['boost time'] = df_all_games['time slow speed'] + df_all_games['time boost speed'] + \
                             df_all_games['time supersonic speed']
df_all_games['ground air time'] = df_all_games['time on ground'] + df_all_games['time low in air'] + \
                                  df_all_games['time high in air']
df_all_games['ball time'] = df_all_games['time in front of ball'] + df_all_games['time behind ball']
df_all_games['position half time'] = df_all_games['time offensive half'] + \
df_all_games['time defensive half']
df_all_games['position third time'] = df_all_games['time offensive third'] + \
df_all_games['time defensive third'] + \
                                      df_all_games['time neutral third']

df_all_games['date'] = pd.to_datetime(df_all_games['date'])

difference(df_all_games, 'date', 'session', session_delay, 0, operator=operator.ge)
df_all_games['goals without assists'] = 0
df_all_games['score O1'] = 0
df_all_games['score O2'] = 0

#df_all_games['session games'] = df.groupby('session')['session'].transform('count')

# Drop useless columns
df_short = df_all_games.drop(columns=['replay title', 'car id', 'car name'])
df_cleaned = df_short
    
# Select df_teams & value in a row
add_df_to_class(player_class_list)
add_df_to_class(team_class_list)

# Quick solution for randoms
player_name_string_list = [player_name_1[0], player_name_1[1], player_name_2[0], player_name_2[1]]
df_randoms = df_cleaned[~df_cleaned['player name'].isin(player_name_string_list)].reset_index(drop='True')

player_class_list[0].df['goals without assists'] = player_class_list[0].df['goals'] - player_class_list[1].df['assists']
player_class_list[1].df['goals without assists'] = player_class_list[1].df['goals'] - player_class_list[0].df['assists']

player_class_list[0].df['score O1'] = df_randoms['score'][::2].reset_index(drop='True')
player_class_list[1].df['score O1'] = df_randoms['score'][::2].reset_index(drop='True')
player_class_list[0].df['score O2'] = df_randoms['score'][1::2].reset_index(drop='True')
player_class_list[1].df['score O2'] = df_randoms['score'][1::2].reset_index(drop='True')

df_player_concat = pd.concat([player_class_list[0].df, player_class_list[1].df])
df_player_concat.sort_values(by=['date', 'player name'], inplace=True)
df_players = df_player_concat.reset_index(drop=True)

# Save players CSV
export_csv = df_players.to_csv(r'Data\Only_OT_2on2.csv', index = None, header=True)



#df_all_games['Goals without Assist'] = player_class_list.0

# # Select df_teams & value in a row
# for team_class in team_class_list:
#     player_class.df = df_cleaned[df_cleaned['player name'].isin(player_class.name)].reset_index(drop='True')
#     value_in_a_row(player_class.df, 'result', 'wins in a row', 1, 1, operator.eq)
#     value_in_a_row(player_class.df, 'result', 'losses in a row', 0, 1, operator.eq)
#     value_in_a_row(player_class.df, 'goals', 'goals in a row', 1, 'df', operator.ge)
#     value_in_a_row(player_class.df, 'saves', 'saves in a row', 1, 'df', operator.ge)
#     value_in_a_row(player_class.df, 'assists', 'assists in a row', 1, 'df', operator.ge)
    
# if ((len(player_class_list[0].df) != len(player_class_list[1].df)) | 
#     (len(player_class_list[0].df) != len(player_class_list[2].df)) | 
#     (len(player_class_list[0].df) != len(player_class_list[3].df))):
#     print("Error: Seems like you forgot a player name.")
#     exit()
# else:
#     pass
    

    
# # Select df_teams
# df_Cloudwalker_Demigod_Hope = df_cleaned[df_cleaned['team name'].isin(team_name_1)].reset_index(drop='True')
# df_Darkhubert_kogut59 = df_cleaned[df_cleaned['team name'].isin(team_name_2)].reset_index(drop='True')
# player_class_list[1].df_kogut59 = df_cleaned[df_cleaned['team name'].isin(team_name_3)].reset_index(drop='True')
# player_class_list[2].player_class_list[3].df = df_cleaned[df_cleaned['team name'].isin(team_name_4)].reset_index(drop='True')
# player_class_list[3].df_Demigod_Hope = df_cleaned[df_cleaned['team name'].isin(team_name_5)].reset_index(drop='True')
# player_class_list[2].df_kogut59 = df_cleaned[df_cleaned['team name'].isin(team_name_6)].reset_index(drop='True')
      
# if ((len(player_class_list[2].df_Demigod_Hope) != len(player_class_list[3].df_kogut59)) | 
#    (len(player_class_list[1].df_kogut59) != len(player_class_list[2].player_class_list[3].df)) | 
#    (len(player_class_list[3].df_Demigod_Hope) != len(player_class_list[2].df_kogut59))):
#     print("Error: Something went wrong with the dataframe team assignment.")
#     exit()
# else:
#     pass



Error: Something went wrong with the dataframe for this class assignment.


## Map Statistics

In [ ]:
maps_list = np.unique(df_all_games.map)
unique_maps_list = int(len(maps_list))
unique_maps_counter = []
index = np.arange(unique_maps_list) 

for i, unique_map in enumerate(maps_list):
    unique_maps_counter.append(int(len(df_all_games[df_all_games.map == unique_map])/4))

fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
rects = ax.bar(index, unique_maps_counter, width, color='g', ecolor='black',)
ax.set_ylabel('Number of Games')
plt.xticks(index, maps_list)
#Title = "Maps played"
#ax.set_title(Title)
#ax.yaxis.grid(True)
plt.xticks(rotation='vertical')
plt.tight_layout()
fig_name = "Map_stats.png"
plt.savefig(plot_path+fig_name)
plt.show()

print("Summary:")
most_played_index = unique_maps_counter.index(max(unique_maps_counter))
print("Most games were played on the map: '" + str(maps_list[most_played_index]) + "'.")
print("Games on none standard maps (e.g. 'Pillars') are removed for the following analysis.")

## Game Length

Since this is not a stored feature, it must be reconstruced. Preferably by using different methods.
The first method is to add up time related features. We can choose between boost time, time spend on ground/in the air,
time in front or behind the ball, time spend in each half of the field or time spend in each third of the field. 
Besides a rounding error, all of these features will give the same answer for one player. 
But one will also find different values for the players in one game. A difference of multiple seconds can be explained 
by demos. Furthermore the difference in time due to demos may vary if the demo happened right before the kickoff-respawn 
starts. However, demos do not explain all discrepancies. A difference of a few seconds can be found in games with no 
demolotion.

The table below illustrates that. Each game takes up 4 rows, so game 1 goes from 0-3, game 2 from 4-7, etc. .

In [ ]:
print(df_all_games[['player name', 'boost time', 'ball time', 'demos taken']][:4])
print("---"*20)
print(df_all_games[['player name', 'boost time', 'ball time', 'demos taken']][4:8])
print("---"*20)
print(df_all_games[['player name', 'boost time', 'ball time', 'demos taken']][8:12])
print("---"*20)
print(df_all_games[['player name', 'boost time', 'ball time', 'demos taken']][200:204])

I assume this due to the way time is counted each player. Basically, the replay file holds a record of where everything 
was spawned. From there it has a list of where everything is going. For that to work, the object (ball, car or boost) has 
to be present on the playfield. If a car was demolished, the replay file will not hold any information of that car for the 
respawn duration and then starts a new list. But this is also the case when a player joins a team, as the player's car will 
be spawned on the field and he has to wait for all players to join so the game can start. 

So my guess ist that the calculated time represents the duration where the car of the player was present on the playfield.
Hence the best approximation of the game duration is to find the lowest time of the player with no demos taken.

The second method would be to use the 'date' feature. Assuming that there is no time delay between two games, you can find
game duration of a game by taking the difference of the 'date' features. This will exclude the very first game of a session
though.

I will stick with the first method. 

In [ ]:
time_list = []

for i in range(len(player_class_list[0].df)):
    player_time = []
    if player_class_list[0].df['demos taken'][i] == 0:
        player_time.append(player_class_list[0].df['boost time'][i])
    if player_class_list[1].df['demos taken'][i] == 0:
        player_time.append(player_class_list[1].df['boost time'][i])
    if len(player_time) >= 1:
        time_list.append(min(player_time)/60)

print("Out of all " + str(len(player_class_list[0].df)) + " games, " + str(len(time_list)) + \
      " meet the requirement of at least one player with no demos taken.")

# Plot  
fig = plt.figure(figsize=(14,8))
plt.hist(time_list, bins=40)
plt.hist(np.mean(time_list), bins=40)
plt.xlabel('Minutes')
plt.ylabel('# Of Games')
#Title = 'Game Duration'
#plt.title('Title)
fig_name = "Game_Duration.png"
plt.savefig(plot_path+fig_name)
plt.show()

print("Summary:")
print("(For the targeted audiance, I'm using the ISO 8601 standard)")
if stats.sem(time_list)*60 < 10:
    print("Average game duration: (" + str(int(np.mean(time_list))) + ":" + \
          str(int((np.mean(time_list)-int(np.mean(time_list)))*60)) + " +/- " + \
          str(int(stats.sem(time_list))) + ":0" + \
          str(int((stats.sem(time_list)-int(stats.sem(time_list)))*60)) + "(stat) + 0:02(sys)) min")
else:
    print("Average game duration: (" + str(int(np.mean(time_list))) + ":" + \
          str(int((np.mean(time_list)-int(np.mean(time_list)))*60)) + " +/- " + \
          str(int(stats.sem(time_list))) + ":" + \
          str(int((stats.sem(time_list)-int(stats.sem(time_list)))*60)) + "(stat) + 0:02(sys)) min")
print("Record shortest game:   " + str(int(min(time_list))) + ":" + \
      str(int((min(time_list)-int(min(time_list)))*60)) + " min")
print("Record longest game:   " + str(int(max(time_list))) + ":" + \
      str(int((max(time_list)-int(max(time_list)))*60)) + " min")

## Session Length

Now let's see how many sessions and how many games were played on average each time. 

In [ ]:
# time_difference = np.ediff1d(player_class_list[0].df.date)
# sesson_index_list = [0]
# for i, time_diff in enumerate(time_difference):
#     if time_diff > session_delay:
#         sesson_index_list.append(i+1)
        
# sesson_index_list.append(player_class_list[0].df.index[-1]+1)
# session_games = np.ediff1d(sesson_index_list)
# index_session = np.arange(len(session_games))
# print("A total of " + str(len(session_games)) + " sessions were found.")

# session_inds = np.unique(player_class_list[0].df.groupby('session')['session'].transform('count'),
#                          return_index=True)[1]
# session_games_2 = ([player_class_list[0].df.groupby('session')['session'].transform('count')[index] 
#                  for index in sorted(indexes)])


df_session_games = player_class_list[0].df.groupby('session', as_index=False)['session games'].sum()
session_games = np.sqrt(df_session_games['session games'].values.tolist())
#index_session = np.arange(len(session_games))
index_session = np.unique(player_class_list[0].df.session)
print("A total of " + str(len(session_games)) + " sessions were found.")  
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
rects = ax.bar(index_session, session_games, width, color='g', ecolor='black',)
ax.set_ylabel('Number of Games')
ax.set_xlabel('Session Number')
ax.axhline(np.mean(session_games), index_session[0]-1,index_session[-1]+1, c="g")
plt.xticks(index_session, index_session+1)
#Title = "Games played per Session"
#ax.set_title(Title)
#ax.yaxis.grid(True)
plt.tight_layout()
fig_name = "Session_Games_Stats.png"
plt.savefig(plot_path+fig_name)
plt.show()

print("Summary:")
print("Average games played with STD per session: " + str(round(np.mean(session_games), 1)) + \
      " +/- " + str(round(np.std(session_games), 1)))
print("Longest session is " + str(int(max(session_games))) + " games long.")
print("Shortes session is " + str(int(min(session_games))) + " games long.")

## Part 2: Session Champions and Records

In [ ]:
session_feature_mean(player_class_list, 'result', 'session winrate', percentage=True)
session_champions_list = []
session_champ(player_class_list, session_champions_list)    

fig_name = "Session_Winrates_Players.png"
session_feature_plot(player_class_list,
                     feature = 'session winrate',
                     ylabel = 'Winrate [%]',
                     fig_name = fig_name,
                     percentage='True')


print("Summary:")

session_wins_list = []
session_draws_list = []
player_names_list = []
for player in player_class_list:
    session_wins_list.append(player.session_result.count(1))
    session_draws_list.append(player.session_result.count(0.5))
    player_names_list.append(player.name[0])
session_max = max(session_wins_list)

if session_wins_list.count(session_max) > 1:
    print("Multiple players managed to solo-win a total of " + str(session_max) + " sessions!")
    print("The record is thus shared between:" )
    for player in player_class_list:
        if player.session_result.count(1) == session_max:
            print(str(player.name[0]))
else:
    for player in player_class_list:
        if player.session_result.count(1) == session_max:
            print("The record of " + str(session_max) + \
                  " sessions-solo-wins is hold by: " + str(player.name[0]))

            
# print(any(sum(1 for _ in islice(g, len(index_session))) == len(index_session) 
#           for k, g in groupby(player_class_list[2].session_result) if k))
                       
# champion_in_a_row_counter = [0, 0, 0, 0]
# champion_in_a_row_index = [0, 0, 0, 0]
# for i in range(len(session_champions_list)-1):
#     if ((session_champions_list[i] == session_champions_list[i+1]) & 
#         (session_champions_list[i] != 'DRAW')):
#         champion_in_a_row_counter[player_names_list.index(session_champions_list[i])] += 1
        
# champion_in_a_row_max = max(champion_in_a_row_counter)
# max_inds = [i for i, x in enumerate(champion_in_a_row_counter) if x == champion_in_a_row_max]
# for k, ind in enumerate(max_inds):
#         champion_in_a_row_index[ind] += 1

# if sum(champion_in_a_row_index) > 1:
#     print("The following player have won " + str(champion_in_a_row_max) + " sessions in a row:")
#     for k, ind in enumerate(champion_in_a_row_index):
#         if ind > 0:
#             print(str(player_names_list[k]))
# else:
#     for k, ind in enumerate(champion_in_a_row_index):
#         if ind > 0:
#             print("A total of " + str(champion_in_a_row_max) + \
#                   " sessions were solo-won in a row by: " + str(player_names_list[k]))

# winrate_overall_counter = [0] * len(player_class_list)
# winrate_overall_kogut59 = round(100*player_class_list[0].df.result.sum()/len(player_class_list[0].df), 2)
# winrate_overall_Demigod_Hope = round(100*player_class_list[1].df.result.sum()/len(player_class_list[1].df), 2)
# winrate_overall_Cloudwalker = round(100*player_class_list[2].df.result.sum()/len(player_class_list[2].df), 2)
# winrate_overall_DarkHubert = round(100*player_class_list[3].df.result.sum()/len(player_class_list[3].df), 2)
# winrate_overall_list = [winrate_overall_kogut59, winrate_overall_Demigod_Hope, winrate_overall_Cloudwalker, winrate_overall_DarkHubert]
# winrate_overall_max = max(winrate_overall_list)
# max_inds = [i for i, x in enumerate(winrate_overall_list) if x == winrate_overall_max]
# for k, ind in enumerate(max_inds):
#         winrate_overall_counter[ind] += 1

# if sum(winrate_overall_counter) > 1:
#     print("The following players have achieved the highest overall winrate of " + str(winrate_overall_max) + "% :")
#     for k, ind in enumerate(winrate_overall_counter):
#         if ind > 0:
#             print(str(player_names_list[k]))
# else:
#     for k, ind in enumerate(winrate_overall_counter):
#         if ind > 0:
#             print("The highest overall winrate of " + str(winrate_overall_max) + "% was achieved by: " + str(player_names_list[k]))

In [ ]:
# Most Goals, Saves, Assists....
goals_and_such_list = ['goals', 'assists', 'saves', 'shots', 'demos inflicted', 'demos taken', 
                       'shots conceded', 'goals conceded', 'goals conceded while last defender', 
                       'goals in a row', 'saves in a row', 'assists in a row', 'wins in a row', 
                       'losses in a row']
fig_name = "Highest_Goals_And_More.png"
most_feature_plot(player_class_list, goals_and_such_list, fig_name, ylim_flip=False, width=0.2)

print("Summary:")
print("Coming Soon.")

In [ ]:
# Most Boost Stuff
most_bosst_list = ['count collected big pads', 'count collected small pads', 'count stolen big pads', 
                     'count stolen small pads', '0 boost time', 'avg boost amount', 
                     'percentage slow speed', 'percentage boost speed',
                     'percentage supersonic speed', 'percentage on ground', 'percentage low in air', 
                     'percentage high in air']

fig_name = "Highest_Boost_Stuff.png"
most_feature_plot(player_class_list, most_bosst_list, fig_name, ylim_flip=False, vertical_flip=False,
                  width=0.2, logscale=True)

print("Summary:")
print("Coming Soon.")

In [ ]:
# def session_feature_mean(class_list, feature, new_column_name, percentage=False):
#     for class_item in class_list:
#         class_item.session_winrate = []
#         class_item.session_result = [0] * len(index_session)
#         for session in range(min(class_item.df.session), max(class_item.df.session)+1):
#             if percentage:
#                 session_feature_avg = 100*class_item.df[feature][class_item.df.session == session].mean()
#             else:
#                 session_feature_avg = class_item.df[feature][class_item.df.session == session].mean()
#             class_item.session_winrate.append(session_feature_avg)
#             class_item.df[new_column_name][class_item.df.session == session] = session_feature_avg


session_feature_mean(team_class_list, 'result', 'session winrate', percentage=True)          
#session_champions_list = []
#session_champ(team_class_list, session_champions_list)

fig_name = "Session_Winrates_Teams.png"
session_feature_plot(team_class_list,
                     feature = 'session winrate',
                     ylabel = 'Winrate [%]',
                     fig_name = fig_name,
                     width=0.1,
                     percentage='True')

print("Summary:")


# for team_class in team_class_list:
#     team_class.session_winrate = []
#     team_class.session_result = [0] * len(index_session)
#     for k in range(len(sesson_index_list)-1):
#         winrate = (100*(team_class.df.result[sesson_index_list[k]:sesson_index_list[k+1]]).mean())
#         team_class.session_winrate.append(winrate)
#         team_class.df.loc[sesson_index_list[k]:sesson_index_list[k+1], 'session winrate'] = winrate


In [ ]:
def solution(A):
    # write your code in Python 3.6
    my_list = [element for element in A if element > 0]
    if len(my_list) == 0:
        return 1
    elif len(my_list) == 1:
        if my_list[0] != 1:
            return 1
        else:
            return 2
    elif ((len(my_list) > 0) & (min(my_list) != 1)):
        return 1
    my_list.sort()
    for ind in range(len(my_list)-1):
        diff = my_list[ind+1] - my_list[ind]
        if diff > 1:
            return (my_list[ind]+1)
        else:
            pass
    return my_list[-1]+1

A=[1, 2, 3]
solution(A)